In [ ]:
import os
import textwrap
from dotenv import load_dotenv, find_dotenv
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain import PromptTemplate

In [ ]:
load_dotenv(find_dotenv())
embeddings = OpenAIEmbeddings()

## Elegir si enseñar o recordar (evita el uso de tokens en OpenAI)

### 1. Enseñar

In [ ]:
# Carga de documentos y extracción de información
# (asegúrate de que haya PDFs en la carpeta documentos)
documents = DirectoryLoader('./documentos/', glob="./*.pdf", loader_cls=PyPDFLoader).load()
# Tratameinto de caracteres indeseados
for d in documents:
    d.page_content = d.page_content.replace('\n', ' ').replace('\t', ' ')
# Separador de texto (límita el texto para que sea soportado por el LLM)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=200
)
textos = text_splitter.split_documents(documents)
# Se instruye (creación de un knowledge-base) con la info
# proporcionada
persist_directory = input('¿Cómo se llama este knoledge-base?: ') + '_kb'
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=textos, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)
# Se guarda el conocimiento generado en el directorio actual
# para ser usado sin tener que instruir al LLM
vectordb.persist()
vectordb = None
os.system(f'zip -r db.zip ./{persist_directory}')

### 2. Recordar (si y solo si Enseñar=Done)

In [ ]:
# Now we can load the persisted database from disk, and use it as normal. 
persist_directory = input('¿Qué knoledge-base deseas usar?: ') + '_kb'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)
retriever = vectordb.as_retriever(search_kwargs={"k": 5})
#retriever = vectordb.as_retriever()

In [ ]:
# Se especifica el LLM de OpenAI
turbo_llm = ChatOpenAI(
    temperature=0.7,
    model_name='gpt-3.5-turbo-0125'
)
# Se personaliza el LLM
template = """
Eres Prometeo, un asistente personal de lectura que habla Español.

Tu tarea consiste en:

1. Ser carismático y ofrecer información sobre ti y tus funciones.

2. Leer detalladamente la información proporcionada en documentos
de texto en formato PDF, para ser capaz de dar respuestas a cualquier tipo de pregunta,
en especial a preguntas puntuales.

SIEMPRE debes responder con base al contexto proporcionado aqui: {context}.
        
Si no sabes la respuesta, puedes decir: 'No sé'.

NUNCA hables del contexto.

Teniendo lo anterior en cuenta, responde la siguiente pregunta: {question}
"""

prometeo_prompt = PromptTemplate(
    template=template, input_variables=["context", "question"]
)
# Creación de cadena
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm,
                                       chain_type="stuff",
                                       retriever=retriever,
                                       chain_type_kwargs={"prompt": prometeo_prompt},
                                       return_source_documents=True)
# Formato y referencias de las respuestas
def wrap_text_preserve_newlines(text, width=70):
    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\nReferencias:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [ ]:
#Demo break down
query = query#input("Hazme una pregunta: ")
llm_response = qa_chain(query)
llm_response

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [ ]:
#Demo break down
query = query#input("Hazme una pregunta: ")
llm_response = qa_chain(query)
llm_response

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)